## Saving and loading objects

In [6]:
import numpy as np
import json

import sfsimodels as sm

## Define a soil

In [8]:
# Set the void ratio and specific gravity
sl = sm.Soil()
sl.id = 1  # Must set an id before saving
sl.e_curr = 0.7
assert sl.unit_dry_weight is None
sl.specific_gravity = 2.95
assert np.isclose(sl.unit_dry_weight, 17000, rtol=0.01)

# set modulus parameters
g_mod_set = 40e6 
sl.g_mod = g_mod_set
sl.poissons_ratio = 0.4

## Save the soil to file then load it

In [12]:
# Create an output object
ecp_output = sm.Output()

# Add the soil to the output object
ecp_output.add_to_dict(sl)

# Add further info to the output object
ecp_output.name = "a single soil"
ecp_output.units = "N, kg, m, s"
ecp_output.comments = ""

# Export the output object to a json string, which can be saved to file
p_str = json.dumps(ecp_output.to_dict(), skipkeys=["__repr__"], indent=4)

# load the json string using the sfsimodels.load_json method
objs = sm.loads_json(p_str, verbose=0)
loaded_soil = objs['soils'][1]
assert np.isclose(loaded_soil.g_mod, sl.g_mod)
print(sl.g_mod)

40000000.0


## Define a soil profile

In [18]:
# Define another soil
sl_2 = sm.Soil()
sl_2.id = 2
sl_2.phi = 33.
sl_2.cohesion = 50000
sl_2.unit_dry_weight = 16000

# Define a soil profile
sp = sm.SoilProfile()
depth_1 = 0.0  # m (set to top of layer from surface)
depth_2 = 4.0  # m

sp.add_layer(depth_1, sl)
sp.add_layer(depth_2, sl_2)
sp.id = 0
sp.height = 12.0  # m (total height of profile)
sp.gwl = 5.0  # m (ground water level from surface)

## Save and load a soil profile

In [20]:
# re-initiate the output object
ecp_output = sm.Output()

# add the soil profile, it automatically adds the associated soil objects
ecp_output.add_to_dict(sp)

# Add further info to the output object
ecp_output.name = "a soil profile"
ecp_output.units = "N, kg, m, s"
ecp_output.comments = ""

# Export the output object to a json string, which can be saved to file
p_str = json.dumps(ecp_output.to_dict(), skipkeys=["__repr__"], indent=4)

# load the json string using the sfsimodels.load_json method
objs = sm.loads_json(p_str, verbose=0)
loaded_soil_profile = objs['soil_profiles'][0]
assert np.isclose(loaded_soil_profile.height, 12)
assert np.isclose(loaded_soil_profile.layer(1).g_mod, sl.g_mod)
print(loaded_soil_profile.height)

12.0
